<a href="https://colab.research.google.com/github/axjasf/YNAB-Categorizer/blob/main/Copy_of_YNAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Libraries

In [1]:
# Installing libraries
!pip install fuzzywuzzy
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.5 MB/s eta 0:00:00


In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
from ipywidgets import widgets
from IPython.display import display
import json
import io
from fuzzywuzzy import process


## Files

In [3]:
# Folder names and File names

HOME_PATH = "/content/drive/MyDrive/Colab Notebooks/YNAB/"
CONFIG_PATH = HOME_PATH + "/config/"
TRANSACTIONS_PATH = HOME_PATH + "/transactions/"

TRANSACTIONS_FILE = "chase.csv"


In [4]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
### Config Files ###

# Read Grocery keywords
grocery_keywords_df = pd.read_csv(CONFIG_PATH + 'grocery_keywords.csv')
grocery_keywords = grocery_keywords_df.iloc[:, 0].str.lower().tolist()

# Checking, if data is valid
# grocery_keywords_df.head()

In [6]:
### Amazon Orders ###

# Read Amazon Order Headers and Items
amazon_order_headers_df = pd.read_csv(TRANSACTIONS_PATH + 'amazon_order_headers.csv')
amazon_order_items_df = pd.read_csv(TRANSACTIONS_PATH + 'amazon_order_items.csv')

# Checking, if data is valid
# amazon_order_headers_df.head()
# amazon_order_items_df.head()

In [7]:
### Transactions ###

# Read bank transactions
transactions_df = pd.read_csv(TRANSACTIONS_PATH + TRANSACTIONS_FILE)

# Checking, if data is valid
# transactions_df.head()


## Global Variables

In [8]:
### CONSTANTS ###

# Major categories
category= {
    "grocery" : "Food, Sports, Entertainment:Food - Groceries & Errands"
}

# Transactions preparation

## Payee identification

In [17]:
def process_config(config_file, transactions_df):

    def identify_candidates(transactions_df, config):

        def fuzzy_match(transactions_df, patterns, payee_field):
            """Identify all fuzzy matches based on patterns."""
            matched_entries_with_scores = []
            for payee in transactions_df[payee_field].unique():
                for pattern in patterns:
                    score = process.extractOne(pattern, [payee])[1]
                    if score >= threshold:
                        matched_entries_with_scores.append((payee, score))
            return matched_entries_with_scores

        matches = fuzzy_match(transactions_df, patterns, payee_field)

        # Filter out any payees present in the negative_list from the all_matches
        matches = [match for match in matches if match[0] not in negative_list]

        # For the purpose of the widgets, exclude items from the positive list
        widget_matches = [match for match in matches if match[0] not in positive_list]

        # Assign matched_df to all matches
        matched_df = transactions_df[transactions_df[payee_field].isin([match[0] for match in matches])]

        return matches, widget_matches, matched_df

    def create_feedback_widgets(matches_with_scores, transactions_df, matched_payees):

        def process_feedback(entry, action, matches, config):
            """Process user feedback."""
            if action == "Approve":
                config['positive_list'].append(entry)
            elif action == "Reject":
                config['negative_list'].append(entry)
                matches = [match for match in matches if match[0] != entry]
            elif action == "Pending":
                config['pending_list'].append(entry)
                matches = [match for match in matches if match[0] != entry]

            matched_df = transactions_df[transactions_df[payee_field].isin([m[0] for m in matches])]
            return matches, matched_df

        def on_submit(button):
            global matched_payees

            for hbox in widgets_list:
                checkbox, action_dropdown = hbox.children
                entry = checkbox.description.split(" (Score:")[0]

                # If the checkbox is checked, handle feedback
                if checkbox.value:
                    action = action_dropdown.value
                    matched_payees = process_feedback(entry, action, matched_payees)

            with open(CONFIG_PATH + filename, 'w') as file:
                json.dump(config, file, indent=4)

        # Initialize an empty list to store the widgets
        widgets_list = []

        # Filter matches_with_scores based on the threshold
        filtered_matches_with_scores = [(entry, score) for entry, score in matches_with_scores if score >= threshold]

        for entry, score in filtered_matches_with_scores:
            checkbox = widgets.Checkbox(value=False, description=f"{entry} (Score: {score})")

            # Preselect dropdown based on score
            preselected_action = 'Approve'

            action_dropdown = widgets.Dropdown(
                options=['Approve', 'Reject', 'Pending'],
                value=preselected_action,
                description='Action:'
            )

            hbox = widgets.HBox([checkbox, action_dropdown])
            widgets_list.append(hbox)  # Append the HBox to the widgets list
            # display(hbox)

        submit_button = widgets.Button(description="Submit Feedback")
        submit_button.on_click(lambda button: on_submit(button))

        return widgets.VBox(widgets_list + [submit_button])

    # Load the config
    config = json.load(open(CONFIG_PATH + config_file))
    patterns = config["patterns"]
    threshold = config["fuzzy_threshold"]
    payee_field = "Description"
    positive_list = config.get("positive_list",[])
    negative_list = config.get("negative_list", [])

    # Identify potential outliers and matches
    matched_payees, widget_payees, matched_df = identify_candidates(transactions_df, config)
    # Display the feedback widgets for all fuzzy matches above the threshold
    feedback_widgets = create_feedback_widgets(widget_payees, transactions_df, matched_payees)

    display(feedback_widgets)

    return matched_df

In [10]:
class Matcher:

    def __init__(self, config_file, transactions_df):
        self.transactions_df = transactions_df
        self.config = json.load(open(CONFIG_PATH + config_file))
        self.config_file = config_file

        self.patterns = self.config["patterns"]
        self.threshold = self.config["fuzzy_threshold"]
        self.payee_field = "Description"
        self.positive_list = self.config.get("positive_list",[])
        self.negative_list = self.config.get("negative_list", [])
        self.matched_payees = []

    def fuzzy_match(self):
        """Identify all fuzzy matches based on patterns."""
        matched_entries_with_scores = []
        for payee in self.transactions_df[self.payee_field].unique():
            for pattern in self.patterns:
                score = process.extractOne(pattern, [payee])[1]
                if score >= self.threshold:
                    matched_entries_with_scores.append((payee, score))
        return matched_entries_with_scores

    def identify_candidates(self):
        matches = self.fuzzy_match()
        matches = [match for match in matches if match[0] not in self.negative_list]
        widget_matches = [match for match in matches if match[0] not in self.positive_list]
        matched_df = self.transactions_df[self.transactions_df[self.payee_field].isin([match[0] for match in matches])]
        return matches, widget_matches, matched_df

    def process_feedback(self, entry, action):
        """Process user feedback."""
        if action == "Approve":
            self.config['positive_list'].append(entry)
        elif action == "Reject":
            self.config['negative_list'].append(entry)
            self.matched_payees = [match for match in self.matched_payees if match[0] != entry]
        elif action == "Pending":
            self.config['pending_list'].append(entry)
            self.matched_payees = [match for match in self.matched_payees if match[0] != entry]

    def create_feedback_widgets(self, matches_with_scores):
        widgets_list = []
        filtered_matches_with_scores = [(entry, score) for entry, score in matches_with_scores if score >= self.threshold]
        for entry, score in filtered_matches_with_scores:
            checkbox = widgets.Checkbox(value=False, description=f"{entry} (Score: {score})")
            preselected_action = 'Approve'
            action_dropdown = widgets.Dropdown(
                options=['Approve', 'Reject', 'Pending'],
                value=preselected_action,
                description='Action:'
            )
            hbox = widgets.HBox([checkbox, action_dropdown])
            widgets_list.append(hbox)

        submit_button = widgets.Button(description="Submit Feedback")
        submit_button.on_click(self.on_submit)
        return widgets.VBox(widgets_list + [submit_button])

    def on_submit(self, button):
        for hbox in self.feedback_widgets.children[:-1]:  # Exclude the submit button
            checkbox, action_dropdown = hbox.children
            entry = checkbox.description.split(" (Score:")[0]
            if checkbox.value:
                action = action_dropdown.value
                self.process_feedback(entry, action)
        with open(CONFIG_PATH + self.config_file, 'w') as file:
            json.dump(self.config, file, indent=4)

    def process(self):
        self.matched_payees, widget_payees, matched_df = self.identify_candidates()
        self.feedback_widgets = self.create_feedback_widgets(widget_payees)
        display(self.feedback_widgets)
        return matched_df


In [11]:
configs_dict = {
    "gas_stations": "gas_stations_patterns_config.json",
    "amazon": "amazon_patterns_config.json",
    "restaurants": "restaurants_patterns_config.json"
}
all_matched_dfs = {}

In [12]:
# Gas Stations
matcher = Matcher(configs_dict['gas_stations'], transactions_df)
matched_df = matcher.process()

In [13]:
# Amazon
matcher = Matcher(configs_dict['amazon'], transactions_df)
matched_df = matcher.process()

In [14]:
# Restaurants
matcher = Matcher(configs_dict['restaurants'], transactions_df)
matched_df = matcher.process()

# Amazon Orders

## Pricing

In [ ]:
### PRICE TOTALS CALCULATIONS ###
### HEADER ###

# Extract the individual payments for one row and put them into columns
def extract_individual_payments_into_columns(df, column_name):
    # Extract dollar values from the specified column
    payments_dollar_values = df[column_name].str.findall(r'\$(\d+\.\d+)')

    # Determine the maximum number of payments in any row
    max_payments = payments_dollar_values.apply(len).max()

    # Pad the lists to ensure they all have a length of max_payments
    padded_payments = payments_dollar_values.apply(lambda x: x + [None] * (max_payments - len(x)))

    # Convert the padded lists to a DataFrame with dynamic column names
    payment_columns = [f'payment_{i+1}' for i in range(max_payments)]
    df[payment_columns] = pd.DataFrame(padded_payments.tolist(), index=df.index)

    # Convert string values to float for further analysis if needed
    for col in payment_columns:
        df[col] = df[col].astype(float)

# Apply the function to the original dataframe
extract_individual_payments_into_columns(amazon_order_headers, "payments")

# amazon_order_headers.head()

In [ ]:
### PRICE TOTALS CALCULATIONS ###
### ITEMS ###

# Item Totals (Quantity * Price) #
amazon_order_items = amazon_order_items.copy()
if amazon_order_items['price'].dtype == 'object':
    amazon_order_items['price'] = pd.to_numeric(amazon_order_items['price'].str.replace('[\$,]', '', regex=True), errors='coerce')
amazon_order_items['quantity'] = amazon_order_items['quantity'].replace([' ', ''], np.nan)
amazon_order_items['quantity'] = pd.to_numeric(amazon_order_items['quantity'], errors='coerce').fillna(1)
amazon_order_items['total'] = amazon_order_items['price'] * amazon_order_items['quantity']

### Distribution of taxes etc. to individual order items ###

# Group the items dataframe by 'order id' and compute the sum of the 'total' column for each order
# Merge the summed items total with the headers dataframe on 'order id'
# Convert 'total_header' column to numeric format, setting non-numeric values to NaN
# Compute the difference between the 'total' from the headers dataframe and the computed sum of items for each order
# Compute the count of items for each order
# Merge the count of items with the merged dataframe
# Compute the 'overall' value for each order
# Merge the 'overall' value with the original items dataframe to create the 'overall' column

items_sum = amazon_order_items.groupby('order id')['total'].sum().reset_index()
merged_df = amazon_order_headers.merge(items_sum, on='order id', how='left', suffixes=('_header', '_items'))
merged_df['total_header'] = pd.to_numeric(merged_df['total_header'].str.replace('[\$,]', '', regex=True), errors='coerce')
merged_df['difference'] = merged_df['total_header'] - merged_df['total_items']
items_count = amazon_order_items.groupby('order id').size().reset_index(name='count')
merged_df = merged_df.merge(items_count, on='order id', how='left')
merged_df['residual'] = merged_df['difference'] / merged_df['count']
amazon_order_items = amazon_order_items.merge(merged_df[['order id', 'residual']], on='order id', how='left')
amazon_order_items['grand_total'] = amazon_order_items['total'] + amazon_order_items['residual'].round(2)

amazon_order_items.head()

## Item categorization

In [ ]:
# Initialize a category column with default value as 'non-grocery'
amazon_order_items['category'] = 'n/a'

# Check each item description against the keywords
for keyword in grocery_keywords:
    amazon_order_items.loc[amazon_order_items['description'].str.lower().str.contains(keyword, na=False), 'category'] = 'grocery'

amazon_order_items.head()


# Output

## Transaction creation

In [ ]:
### SPLIT TRANSACTION ###

def create_split(category, memo, amount):
    return (category, memo, amount)

def create_transaction(date, total_amount, payee, memo, category=None, splits=None):
    return {
        "date": date,
        "total_amount": total_amount,
        "payee": payee,
        "memo": memo,
        "category": category,
        "splits": splits
    }

def export_qif_transactions(filename, transactions):
    qif_content = "!Type:Cash\n"

    for transaction in transactions:
        # Add the transaction details
        qif_content += f"D{transaction['date']}\n"
        qif_content += f"T{transaction['total_amount']}\n"
        qif_content += f"P{transaction['payee']}\n"
        qif_content += f"M{transaction['memo']}\n"

        # If a category is provided for a simple transaction, add it
        if transaction.get('category') and not transaction.get('splits'):
            qif_content += f"L{transaction['category']}\n"

        # If splits exist for the transaction, add them
        splits = transaction.get('splits')
        if splits:
            qif_content += "LSplit Transaction\n"
            for split in splits:
                category, memo, amount = split
                qif_content += f"S{category}\n"
                qif_content += f"E{memo}\n"
                qif_content += f"${amount}\n"

        # End the transaction
        qif_content += "^\n"

    # Export to a file in Google Drive
    with open(HOME_PATH + "/results/" + filename, "w") as file:
        file.write(qif_content)

def export_qif_transactions_by_chunk(transactions, x, base_filename="transactions"):
    """
    Export the transactions in QIF format, splitting them into separate files,
    each containing up to x transactions.

    Parameters:
    - transactions: List of transaction dictionaries.
    - x: Number of transactions per file.
    - base_filename: Base name for the output files. Files will be named as base_filename_1.qif, base_filename_2.qif, ...

    Returns:
    - List of saved file paths.
    """

    # Validate x
    if not (1 <= x <= 100):
        raise ValueError("x should be between 1 and 100")

    saved_files = []

    for idx, i in enumerate(range(0, len(transactions), x)):
        chunk = transactions[i:i+x]
        qif_content = "!Type:Cash\n"

        for transaction in chunk:
            # Add the transaction details
            qif_content += f"D{transaction['date']}\n"
            qif_content += f"T{transaction['total_amount']}\n"
            qif_content += f"P{transaction['payee']}\n"
            qif_content += f"M{transaction['memo']}\n"

            # If a category is provided for a simple transaction, add it
            if transaction.get('category') and not transaction.get('splits'):
                qif_content += f"L{transaction['category']}\n"

            # If splits exist for the transaction, add them
            splits = transaction.get('splits')
            if splits:
                qif_content += "LSplit Transaction\n"
                for split in splits:
                    category, memo, amount = split
                    qif_content += f"S{category}\n"
                    qif_content += f"E{memo}\n"
                    qif_content += f"${amount}\n"

            # End the transaction
            qif_content += "^\n"

        # Define filename for this chunk
        filename = f"{base_filename}_{idx+1}.qif"
        filepath = HOME_PATH + "/results/" + filename

        # Export to a file
        with open(filepath, "w") as file:
            file.write(qif_content)

        saved_files.append(filepath)

    return saved_files


## QIF File creation

In [ ]:
# Using the provided functions to regenerate the QIF transactions and export them

# Generate the QIF transactions

transactions = []

from datetime import datetime

# Iterate over each unique order
for order_id, group in amazon_order_items.groupby('order id'):
    order_date = group['order date'].iloc[0]
    total_amount = group['grand_total'].sum()
    payee = "Amazon" + " " + order_id

    if len(group) == 1: # Check for single item transactions
        row = group.iloc[0]
        if row['category'] == 'grocery':
            transaction = create_transaction(order_date, f"{row['grand_total']:.2f}", "Amazon " + order_id, "", category=category["grocery"])
        else:
            transaction = create_transaction(order_date, f"{total_amount:.2f}", payee, memo=row['description'])
        transactions.append(transaction)

    else:
        # For multiple items in the transaction:
        # Grocery split amount
        grocery_total = group.loc[group['category'] == 'grocery', 'grand_total'].sum()

        # Non-grocery splits
        non_grocery_splits = []
        for _, row in group[group['category'] != 'grocery'].iterrows():
          # Ensure row has valid data
          if 'description' in row and 'grand_total' in row:
            split = create_split("", row['description'], f"{row['grand_total']:.2f}")
            non_grocery_splits.append(split)

        # If grocery total is not zero, add it as a split
        if grocery_total != 0:
            grocery_split = create_split(category["grocery"], "", f"{grocery_total:.2f}")
            non_grocery_splits.append(grocery_split)

        # Create the transaction
        transaction = create_transaction(order_date, f"{total_amount:.2f}", "Amazon " + order_id, "", splits=non_grocery_splits)
        transactions.append(transaction)

# Export the transactions to a QIF file by chunks
export_qif_transactions_by_chunk(transactions, 100)
